In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import nltk

import re
import requests
import pandas as pd
import numpy as np
import json
import os
import dotenv
import sys
import lyricsgenius
import base64
from glob import glob
sys.tracebacklimit = 0 # turn off the error tracebacks
import plotly_express as px
#from lib.textparser import TextParser
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from numpy.linalg import norm
from scipy.spatial.distance import pdist
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from scipy.linalg import eigh
from sklearn.decomposition import PCA
import seaborn as sns

In [2]:
OHCO = ['decade_id', 'song_num', 'stanza_num', 'line_num', 'token_num']

In [3]:
TFIDF = pd.read_csv('data/derived_tables/TFIDF')
TFIDF

,decade_id,1,10k,110th,1k,1s,2,220kid,22nd,24hoursav,...,детство,еlla,еsta,еyes,ẹ,ọmọ,ọpẹ,在中間,站在中間,박혜진
0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.0,0.000000,0.000000,0.001722,0.000000,0.001559,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001722,0.003444,0.000000
2,2,0.0,0.002508,0.000000,0.000000,0.000000,0.003690,0.001254,0.001254,0.001254,...,0.000000,0.001254,0.001254,0.001254,0.002508,0.001254,0.001254,0.000000,0.000000,0.001254
3,3,0.0,0.000000,0.001463,0.000000,0.000000,0.000540,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,0.0,0.000000,0.001329,0.000000,0.002167,0.000000,0.000000,0.000000,0.000000,...,0.001329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000777,0.000000,0.000000,0.000000,...,0.001052,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
LIB = pd.read_csv('data/core_tables/LIB')
LIB

,decade_id,Decade,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,source_file_path,song_regex
0,0,2000s,0.596700,0.758047,-5.768187,0.068727,0.106050,0.043453,0.188454,0.542946,124.718867,../data/decades/2000s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
1,1,2010s,0.636427,0.685193,-5.951373,0.065150,0.170150,0.018582,0.176034,0.470102,119.749207,../data/decades/2010s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
2,2,2020s,0.679820,0.630467,-6.311587,0.080545,0.264939,0.008440,0.177664,0.553668,122.403027,../data/decades/2020s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
3,3,70s,0.570180,0.606453,-9.802827,0.048952,0.310996,0.031435,0.170771,0.624508,121.161153,../data/decades/70s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
4,4,80s,0.619993,0.690873,-8.911047,0.048221,0.174174,0.049269,0.153783,0.662967,121.739340,../data/decades/80s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
5,5,90s,0.571880,0.721047,-7.519947,0.047557,0.116097,0.076750,0.171184,0.569021,117.320680,../data/decades/90s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]


In [5]:
DOC = TFIDF.reset_index()[['decade_id']]
DOC = DOC.merge(LIB, on='decade_id', how='left')
DOC.set_index(['decade_id'],inplace=True)
DOC

,Decade,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,source_file_path,song_regex
decade_id,,,,,,,,,,,,
0,2000s,0.596700,0.758047,-5.768187,0.068727,0.106050,0.043453,0.188454,0.542946,124.718867,../data/decades/2000s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
1,2010s,0.636427,0.685193,-5.951373,0.065150,0.170150,0.018582,0.176034,0.470102,119.749207,../data/decades/2010s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
2,2020s,0.679820,0.630467,-6.311587,0.080545,0.264939,0.008440,0.177664,0.553668,122.403027,../data/decades/2020s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
3,70s,0.570180,0.606453,-9.802827,0.048952,0.310996,0.031435,0.170771,0.624508,121.161153,../data/decades/70s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
4,80s,0.619993,0.690873,-8.911047,0.048221,0.174174,0.049269,0.153783,0.662967,121.739340,../data/decades/80s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]
5,90s,0.571880,0.721047,-7.519947,0.047557,0.116097,0.076750,0.171184,0.569021,117.320680,../data/decades/90s.txt,\[Trackname:\s[^\]]+\]\s\[Artist:\s[^\]]+\]


# PCA

In [6]:
def get_PCA(X, k, norm_docs = True, center_by_mean = True, center_by_variance = True):
    if norm_docs:
        X = (X.T / np.linalg.norm(X, 2, axis=1)).T
    
    if center_by_mean:
        X = X - X.mean(axis=0)
    
    if center_by_variance:
        X = X / X.std(axis=0)

    pca = PCA(n_components=k)
    pca.fit(X)
    
    loadings = pd.DataFrame(pca.components_.T, index=X.columns)
    loadings.columns = [f'PC{i+1}' for i in range(k)]
    
    dcm = pd.DataFrame(pca.transform(X), index=X.index)
    dcm.columns = [f'PC{i+1}' for i in range(k)]
    
    comp = pd.DataFrame({
        'EVR': pca.explained_variance_ratio_,
        'CEVR': np.cumsum(pca.explained_variance_ratio_)
    }, index=[f'PC{i+1}' for i in range(k)])
    
    return loadings, dcm, comp

LOADINGS, DCM, COMPINF = get_PCA(X=TFIDF, k=6, norm_docs=True, center_by_mean=False, center_by_variance=False)
#DCM = DOC.join(DCM)


### Top 5 Loadings for 1st Component

In [7]:
LOADINGS['PC1'].sort_values(ascending=False).to_frame().head(5)

,PC1
aye,0.215730
ashake,0.193604
tonights,0.143820
beep,0.134458
boobooooh,0.132757


### Bottom 5 Loadings for 2nd Component

In [8]:
LOADINGS['PC2'].sort_values(ascending=True).to_frame().head(5)

,PC2
jammin,-0.057338
dit,-0.048797
decade_id,-0.039013
chh,-0.038946
chhchh,-0.038946
